# 04 CycleGAN训练实验（Colab版本）

- 从GitHub仓库拉取代码
- CycleGAN模型训练（Label ↔ Photo双向转换）
- 循环一致性损失
- 适配Google Colab环境

## 使用说明

1. 在Colab中打开此notebook
2. 确保已启用GPU运行时（运行时 → 更改运行时类型 → GPU）
3. 按顺序运行所有单元格
4. 如果需要上传数据，请按照提示操作


## 1. 环境设置和GitHub仓库克隆

**重要提示**：
- Checkpoint文件较大，建议保存到Google Drive
- 代码已配置为自动将checkpoint保存到Drive
- 恢复训练时，只需设置 `RESUME_EPOCH` 参数即可


In [ ]:
# ============================================
# Colab环境设置 - 从GitHub克隆仓库
# ============================================

import os
from pathlib import Path
import sys
import subprocess

# 检测是否在Colab环境中
try:
    import google.colab
    IN_COLAB = True
    print("✅ 检测到Colab环境")
except ImportError:
    IN_COLAB = False
    print("⚠️  未检测到Colab环境，将使用本地路径")

if IN_COLAB:
    # ============================================
    # 方法1：从GitHub克隆仓库（推荐）
    # ============================================
    # 请修改为你的实际GitHub仓库地址
    GITHUB_REPO_URL = "https://github.com/AltheD/Image-to-Image-Translation-Experiment"
    REPO_NAME = "Image-to-Image-Translation-Experiment"
    
    # 如果仓库是私有的，可以使用以下格式（需要设置GitHub token）：
    # GITHUB_REPO_URL = "https://YOUR_TOKEN@github.com/YOUR_USERNAME/Image-to-Image-Translation-Experiment.git"
    
    USE_GITHUB_CLONE = True  # 设置为True以启用GitHub克隆
    
    if USE_GITHUB_CLONE and GITHUB_REPO_URL != "https://github.com/YOUR_USERNAME/Image-to-Image-Translation-Experiment.git":
        print("正在从GitHub克隆仓库...")
        repo_path = Path(f"/content/{REPO_NAME}")
        if not repo_path.exists():
            try:
                subprocess.run(["git", "clone", GITHUB_REPO_URL, str(repo_path)], check=True)
                print(f"✅ 仓库克隆完成: {repo_path}")
            except subprocess.CalledProcessError as e:
                print(f"❌ 克隆失败: {e}")
                print("   请检查：")
                print("   1. GitHub仓库URL是否正确")
                print("   2. 仓库是否为公开，或已配置访问权限")
                print("   3. 网络连接是否正常")
                USE_GITHUB_CLONE = False
        else:
            print(f"✅ 仓库已存在: {repo_path}")
            # 可选：更新仓库
            # subprocess.run(["git", "-C", str(repo_path), "pull"], check=False)
        
        if USE_GITHUB_CLONE:
            ROOT = repo_path
        else:
            ROOT = Path("/content")
    else:
        # ============================================
        # 方法2：直接使用上传的文件（如果notebook已上传到Colab）
        # ============================================
        ROOT = Path("/content")
        print("⚠️  未配置GitHub仓库URL，使用/content目录")
        print("   请确保项目文件已上传到Colab，或修改上面的GITHUB_REPO_URL")
    
    # 切换到项目根目录
    os.chdir(ROOT)
    
    print(f"\n✅ Colab环境设置完成！")
    print(f"项目根目录：{ROOT}")
    print(f"当前工作目录：{os.getcwd()}")
    
    # 挂载Google Drive（可选，用于保存结果和数据）
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive已挂载")
        DRIVE_ROOT = Path("/content/drive/MyDrive")
    except Exception as e:
        print(f"⚠️  Google Drive挂载失败: {e}")
        DRIVE_ROOT = None
else:
    # 本地环境
    ROOT = Path("..").resolve()
    os.chdir(ROOT)
    print(f"✅ 本地环境设置完成！")
    print(f"项目根目录：{ROOT}")
    DRIVE_ROOT = None

# 保存ROOT路径供后续使用
COLAB_WORK_DIR = ROOT


In [ ]:
# ============================================
# 安装依赖包
# ============================================

if IN_COLAB:
    print("正在安装依赖包...")
    
    # 安装必要的Python包
    packages = [
        "torch",
        "torchvision",
        "tqdm",
        "Pillow",
        "scikit-image",  # 用于SSIM计算
    ]
    
    for package in packages:
        print(f"安装 {package}...")
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], check=False)
    
    print("✅ 依赖包安装完成！")
else:
    print("本地环境，跳过依赖安装")

# 检查GPU可用性
print("\n正在检查GPU...")
try:
    import torch
    print(f"✅ PyTorch 版本: {torch.__version__}")
    
    cuda_available = torch.cuda.is_available()
    if cuda_available:
        print(f"✅ GPU可用: {torch.cuda.get_device_name(0)}")
        print(f"   CUDA版本: {torch.version.cuda}")
        print(f"   GPU数量: {torch.cuda.device_count()}")
        print(f"   GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        print("⚠️  警告：未检测到GPU，将使用CPU运行（速度较慢）")
        print("   请在Colab中启用GPU：运行时 → 更改运行时类型 → GPU")
except ImportError:
    print("⚠️  警告：torch未安装")
except Exception as e:
    print(f"⚠️  检查GPU时出错: {e}")


In [ ]:
# ============================================
# 验证项目文件结构
# ============================================

print("📋 验证项目文件...")
required_files = [
    ("src/models/cyclegan_generator.py", "CycleGAN生成器"),
    ("src/models/cyclegan_discriminator.py", "CycleGAN判别器"),
    ("src/data/dataset.py", "数据集文件"),
    ("src/data/transforms.py", "数据变换文件"),
    ("src/eval/metrics.py", "评估指标"),
    ("data/splits/cityscapes_split_seed42.json", "数据划分文件"),
]

all_ok = True
for file_path, description in required_files:
    full_path = ROOT / file_path
    if full_path.exists():
        print(f"✅ {description}: {file_path}")
    else:
        print(f"❌ {description} 缺失: {file_path}")
        all_ok = False

if all_ok:
    print("\n🎉 所有必需文件已就绪！")
else:
    print("\n⚠️  部分文件缺失，请检查项目结构")
    print("   如果使用GitHub克隆，请确保仓库完整")
    print("   如果直接上传，请确保所有文件都已上传到Colab")


## 2. 数据准备

**重要提示**：Cityscapes数据集较大，需要手动准备：

1. **选项A（推荐）**：如果数据已在Google Drive中
   - 将数据文件夹上传到Google Drive
   - 在下面的代码中指定Drive中的路径

2. **选项B**：如果数据在GitHub仓库中
   - 确保数据已包含在仓库中（可能需要使用Git LFS）

3. **选项C**：在Colab中下载数据
   - 使用wget或其他工具下载数据集
   - 解压到指定目录


In [ ]:
# ============================================
# 数据路径设置
# ============================================

# 方法1：从Google Drive加载数据（推荐）
USE_DRIVE_DATA = False  # 如果数据在Drive中，设置为True
DRIVE_DATA_PATH = "/content/drive/MyDrive/cityscapes_data"  # 修改为你的Drive数据路径

# 方法2：使用Colab本地数据（如果已上传或克隆）
LOCAL_DATA_PATH = ROOT / "data"

# 选择数据路径
if USE_DRIVE_DATA and DRIVE_ROOT and Path(DRIVE_DATA_PATH).exists():
    DATA_ROOT = Path(DRIVE_DATA_PATH)
    print(f"✅ 使用Google Drive数据: {DATA_ROOT}")
elif LOCAL_DATA_PATH.exists():
    DATA_ROOT = LOCAL_DATA_PATH
    print(f"✅ 使用本地数据: {DATA_ROOT}")
else:
    # 创建数据目录结构（用于后续上传）
    DATA_ROOT = ROOT / "data"
    DATA_ROOT.mkdir(parents=True, exist_ok=True)
    (DATA_ROOT / "processed").mkdir(exist_ok=True)
    (DATA_ROOT / "splits").mkdir(exist_ok=True)
    print(f"⚠️  数据目录不存在，已创建: {DATA_ROOT}")
    print("   请将数据上传到此目录，或修改上面的路径设置")

# 检查数据是否存在
SPLIT_INDEX = DATA_ROOT / "splits" / "cityscapes_split_seed42.json"
TRAIN_LABEL_DIR = DATA_ROOT / "processed" / "train" / "label"
TRAIN_PHOTO_DIR = DATA_ROOT / "processed" / "train" / "photo"
VAL_LABEL_DIR = DATA_ROOT / "processed" / "val" / "label"
VAL_PHOTO_DIR = DATA_ROOT / "processed" / "val" / "photo"

print("\n📂 检查数据文件...")
if SPLIT_INDEX.exists():
    print(f"✅ 数据划分文件: {SPLIT_INDEX}")
else:
    print(f"❌ 数据划分文件缺失: {SPLIT_INDEX}")

for dir_path, name in [
    (TRAIN_LABEL_DIR, "训练集Label"),
    (TRAIN_PHOTO_DIR, "训练集Photo"),
    (VAL_LABEL_DIR, "验证集Label"),
    (VAL_PHOTO_DIR, "验证集Photo"),
]:
    if dir_path.exists():
        file_count = len(list(dir_path.glob("*")))
        print(f"✅ {name}: {dir_path} ({file_count} 个文件)")
    else:
        print(f"❌ {name} 缺失: {dir_path}")

# 如果数据不存在，提供上传提示
if not all([SPLIT_INDEX.exists(), TRAIN_LABEL_DIR.exists(), TRAIN_PHOTO_DIR.exists()]):
    print("\n" + "="*60)
    print("📤 数据上传提示")
    print("="*60)
    print("如果数据在本地，可以使用以下方法上传到Colab：")
    print("1. 使用Colab的文件上传功能（适合小文件）")
    print("2. 使用Google Drive（推荐，适合大文件）")
    print("3. 使用wget从URL下载（如果有公开链接）")
    print(f"\n数据应放在: {DATA_ROOT}")
    print("="*60)


In [ ]:
# 添加项目根目录到Python路径
sys.path.insert(0, str(ROOT))
print(f"项目根目录：{ROOT}")
print(f"当前工作目录：{Path.cwd()}")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
import json

from src.models.cyclegan_generator import CycleGANGenerator
from src.models.cyclegan_discriminator import CycleGANDiscriminator
from src.data.dataset import CityscapesDataset
from src.data.transforms import build_transform
from src.eval.metrics import evaluate_batch, calculate_psnr, calculate_ssim, calculate_mae

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 输出根目录（在Colab中保存到/content/outputs，也可以保存到Drive）
# 重要：checkpoint保存到Google Drive以确保持久化
USE_DRIVE_FOR_CHECKPOINTS = True  # 设置为True以将checkpoint保存到Drive

if IN_COLAB and DRIVE_ROOT:
    if USE_DRIVE_FOR_CHECKPOINTS:
        # checkpoint保存到Drive（持久化，不会因会话断开而丢失）
        CHECKPOINT_DIR = DRIVE_ROOT / "cyclegan_checkpoints"
        print(f"📁 Checkpoint将保存到Google Drive: {CHECKPOINT_DIR}")
    else:
        # checkpoint保存到本地（临时）
        CHECKPOINT_DIR = ROOT / "outputs" / "checkpoints" / "cyclegan"
        print(f"📁 Checkpoint将保存到本地: {CHECKPOINT_DIR}")
    
    # 其他输出（图像、图表、日志）可以保存到Drive或本地
    OUTPUT_ROOT = DRIVE_ROOT / "cyclegan_outputs" if DRIVE_ROOT else ROOT / "outputs"
    print(f"📁 其他输出将保存到: {OUTPUT_ROOT}")
else:
    OUTPUT_ROOT = ROOT / "outputs"
    CHECKPOINT_DIR = OUTPUT_ROOT / "checkpoints" / "cyclegan"
    print(f"📁 输出将保存到本地: {OUTPUT_ROOT}")

IMAGES_DIR = OUTPUT_ROOT / "images" / "cyclegan"
FIGURES_DIR = OUTPUT_ROOT / "figures" / "cyclegan"
LOGS_DIR = OUTPUT_ROOT / "logs" / "cyclegan"

# 创建目录
for d in [CHECKPOINT_DIR, IMAGES_DIR, FIGURES_DIR, LOGS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("✅ 路径设置完成！")
print(f"   Checkpoint目录: {CHECKPOINT_DIR}")
print(f"   图像目录: {IMAGES_DIR}")
print(f"   图表目录: {FIGURES_DIR}")
print(f"   日志目录: {LOGS_DIR}")


## 4. 测试CycleGAN模型架构


In [ ]:
# 创建模型并测试前向传播
generator_G = CycleGANGenerator(in_channels=3, out_channels=3, n_residual_blocks=9).to(device)
generator_F = CycleGANGenerator(in_channels=3, out_channels=3, n_residual_blocks=9).to(device)
discriminator_photo = CycleGANDiscriminator(in_channels=3).to(device)
discriminator_label = CycleGANDiscriminator(in_channels=3).to(device)

generator_G.eval()
generator_F.eval()
discriminator_photo.eval()
discriminator_label.eval()

# 测试输入
label = torch.randn(1, 3, 256, 256).to(device)
photo = torch.randn(1, 3, 256, 256).to(device)

print("=" * 60)
print("CycleGAN模型架构测试")
print("=" * 60)

with torch.no_grad():
    # 生成器G（Label→Photo）
    fake_photo = generator_G(label)
    print(f"\n生成器G (Label→Photo):")
    print(f"  输入: {label.shape} → 输出: {fake_photo.shape}")
    print(f"  输出范围: [{fake_photo.min():.3f}, {fake_photo.max():.3f}] (期望: [-1, 1])")
    
    # 生成器F（Photo→Label）
    fake_label = generator_F(photo)
    print(f"\n生成器F (Photo→Label):")
    print(f"  输入: {photo.shape} → 输出: {fake_label.shape}")
    print(f"  输出范围: [{fake_label.min():.3f}, {fake_label.max():.3f}] (期望: [-1, 1])")
    
    # 循环一致性测试
    recovered_label = generator_F(fake_photo)
    recovered_photo = generator_G(fake_label)
    print(f"\n循环一致性测试:")
    print(f"  Label→Photo→Label: {label.shape} → {recovered_label.shape}")
    print(f"  Photo→Label→Photo: {photo.shape} → {recovered_photo.shape}")
    
    # 判别器测试
    pred_real_photo = discriminator_photo(photo)
    pred_fake_photo = discriminator_photo(fake_photo)
    pred_real_label = discriminator_label(label)
    pred_fake_label = discriminator_label(fake_label)
    
    print(f"\n判别器D_photo:")
    print(f"  真实photo: {pred_real_photo.shape}")
    print(f"  生成photo: {pred_fake_photo.shape}")
    
    print(f"\n判别器D_label:")
    print(f"  真实label: {pred_real_label.shape}")
    print(f"  生成label: {pred_fake_label.shape}")

# 计算参数量
total_params_G = sum(p.numel() for p in generator_G.parameters())
total_params_F = sum(p.numel() for p in generator_F.parameters())
total_params_D_photo = sum(p.numel() for p in discriminator_photo.parameters())
total_params_D_label = sum(p.numel() for p in discriminator_label.parameters())

print(f"\n参数量统计:")
print(f"  生成器G: {total_params_G:,}")
print(f"  生成器F: {total_params_F:,}")
print(f"  判别器D_photo: {total_params_D_photo:,}")
print(f"  判别器D_label: {total_params_D_label:,}")
print(f"  总参数量: {total_params_G + total_params_F + total_params_D_photo + total_params_D_label:,}")

print("\n✅ CycleGAN模型架构测试通过！")


## 5. 准备数据集


In [ ]:
# CycleGAN数据增强配置（训练集使用强增强，验证集不使用增强）
train_transform = build_transform(
    image_size=256,
    jitter=True,
    horizontal_flip=True,
    color_jitter=(0.2, 0.2, 0.2, 0.05),
    scale_range=(0.8, 1.2),
    normalize_mode="tanh"
)

val_transform = build_transform(
    image_size=256,
    jitter=False,
    horizontal_flip=False,
    color_jitter=None,
    scale_range=None,
    normalize_mode="tanh"
)

# 加载数据集
try:
    train_dataset = CityscapesDataset(
        root=DATA_ROOT,
        split="train",
        split_index=SPLIT_INDEX,
        transform=train_transform
    )
    val_dataset = CityscapesDataset(
        root=DATA_ROOT,
        split="val",
        split_index=SPLIT_INDEX,
        transform=val_transform
    )
    
    # 在Colab中，num_workers设置为0以避免多进程问题
    num_workers = 0
    train_loader = DataLoader(
        train_dataset,
        batch_size=1,  # CycleGAN论文设置batch_size=1
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if device.type == "cuda" else False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if device.type == "cuda" else False
    )
    
    print(f"✅ 数据集加载完成！")
    print(f"  训练集: {len(train_dataset)} 样本")
    print(f"  验证集: {len(val_dataset)} 样本")
    print(f"  Batch size: 1 (CycleGAN论文设置)")
except Exception as e:
    print(f"❌ 数据集加载失败: {e}")
    print("   请确保数据已正确上传到Colab")
    raise


## 6. 定义损失函数和训练函数


In [ ]:
# GAN损失函数（使用MSE Loss替代BCE Loss，更稳定）
class GANLoss(nn.Module):
    """GAN损失函数（使用MSE Loss替代BCE Loss，更稳定）"""
    def __init__(self, target_real_label=1.0, target_fake_label=0.0):
        super(GANLoss, self).__init__()
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))
        self.loss = nn.MSELoss()
    
    def __call__(self, prediction, target_is_real):
        if target_is_real:
            target_tensor = self.real_label
        else:
            target_tensor = self.fake_label
        
        target_tensor = target_tensor.to(prediction.device).expand_as(prediction)
        loss = self.loss(prediction, target_tensor)
        return loss


def save_triplet(label, generated, ground_truth, save_path, title="Label | Generated | Ground Truth"):
    """保存三联图：Label / Generated / Ground Truth"""
    def tensor_to_image(tensor):
        if tensor.dim() == 3:
            img = tensor.permute(1, 2, 0).cpu().numpy()
        else:
            img = tensor[0].permute(1, 2, 0).cpu().numpy()
        if img.min() < 0:
            img = (img + 1) / 2
        img = np.clip(img, 0, 1)
        img = (img * 255).astype(np.uint8)
        return img
    
    label_img = tensor_to_image(label)
    gen_img = tensor_to_image(generated)
    gt_img = tensor_to_image(ground_truth)
    
    triplet = np.hstack([label_img, gen_img, gt_img])
    Image.fromarray(triplet).save(save_path)


def save_cycle_triplet(label, fake_photo, recovered_label, photo, fake_label, recovered_photo, save_path):
    """保存CycleGAN的双向循环结果"""
    def tensor_to_image(tensor):
        if tensor.dim() == 3:
            img = tensor.permute(1, 2, 0).cpu().numpy()
        else:
            img = tensor[0].permute(1, 2, 0).cpu().numpy()
        if img.min() < 0:
            img = (img + 1) / 2
        img = np.clip(img, 0, 1)
        img = (img * 255).astype(np.uint8)
        return img
    
    label_img = tensor_to_image(label)
    fake_photo_img = tensor_to_image(fake_photo)
    recovered_label_img = tensor_to_image(recovered_label)
    photo_img = tensor_to_image(photo)
    fake_label_img = tensor_to_image(fake_label)
    recovered_photo_img = tensor_to_image(recovered_photo)
    
    row1 = np.hstack([label_img, fake_photo_img, recovered_label_img])
    row2 = np.hstack([photo_img, fake_label_img, recovered_photo_img])
    cycle_result = np.vstack([row1, row2])
    Image.fromarray(cycle_result).save(save_path)


print("✅ 损失函数和工具函数定义完成！")


In [ ]:
def train_epoch(
    generator_G, generator_F,
    discriminator_photo, discriminator_label,
    dataloader, gan_loss, l1_loss,
    optimizer_G, optimizer_F,
    optimizer_D_photo, optimizer_D_label,
    device, epoch, lambda_cycle=10.0, lambda_identity=0.5
):
    """训练一个epoch"""
    generator_G.train()
    generator_F.train()
    discriminator_photo.train()
    discriminator_label.train()
    
    total_loss_G = 0.0
    total_loss_F = 0.0
    total_loss_D_photo = 0.0
    total_loss_D_label = 0.0
    total_loss_cycle = 0.0
    num_batches = 0
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch} [Train]")
    for batch_idx, batch in enumerate(pbar):
        label = batch["label"].to(device)
        photo = batch["photo"].to(device)
        
        batch_size = label.size(0)
        
        # ========== 训练判别器 D_photo ==========
        optimizer_D_photo.zero_grad()
        
        real_photo = photo
        pred_real_photo = discriminator_photo(real_photo)
        loss_D_photo_real = gan_loss(pred_real_photo, True)
        
        fake_photo = generator_G(label.detach())
        pred_fake_photo = discriminator_photo(fake_photo.detach())
        loss_D_photo_fake = gan_loss(pred_fake_photo, False)
        
        loss_D_photo = (loss_D_photo_real + loss_D_photo_fake) * 0.5
        loss_D_photo.backward()
        optimizer_D_photo.step()
        
        # ========== 训练判别器 D_label ==========
        optimizer_D_label.zero_grad()
        
        real_label = label
        pred_real_label = discriminator_label(real_label)
        loss_D_label_real = gan_loss(pred_real_label, True)
        
        fake_label = generator_F(photo.detach())
        pred_fake_label = discriminator_label(fake_label.detach())
        loss_D_label_fake = gan_loss(pred_fake_label, False)
        
        loss_D_label = (loss_D_label_real + loss_D_label_fake) * 0.5
        loss_D_label.backward()
        optimizer_D_label.step()
        
        # ========== 训练生成器 G 和 F ==========
        optimizer_G.zero_grad()
        optimizer_F.zero_grad()
        
        # 对抗损失
        fake_photo = generator_G(label)
        pred_fake_photo = discriminator_photo(fake_photo)
        loss_GAN_G = gan_loss(pred_fake_photo, True)
        
        fake_label = generator_F(photo)
        pred_fake_label = discriminator_label(fake_label)
        loss_GAN_F = gan_loss(pred_fake_label, True)
        
        # 循环一致性损失
        recovered_label = generator_F(fake_photo)
        loss_cycle_forward = l1_loss(recovered_label, label) * lambda_cycle
        
        recovered_photo = generator_G(fake_label)
        loss_cycle_backward = l1_loss(recovered_photo, photo) * lambda_cycle
        
        # 身份损失（可选）
        if lambda_identity > 0:
            identity_photo = generator_G(photo)
            loss_identity_G = l1_loss(identity_photo, photo) * lambda_identity
            
            identity_label = generator_F(label)
            loss_identity_F = l1_loss(identity_label, label) * lambda_identity
        else:
            loss_identity_G = torch.tensor(0.0, device=device)
            loss_identity_F = torch.tensor(0.0, device=device)
        
        # 重新计算以分离计算图
        fake_label_detached = fake_label.detach()
        recovered_photo_G = generator_G(fake_label_detached)
        loss_cycle_backward_G = l1_loss(recovered_photo_G, photo) * lambda_cycle
        
        fake_photo_detached = fake_photo.detach()
        recovered_label_F = generator_F(fake_photo_detached)
        loss_cycle_forward_F = l1_loss(recovered_label_F, label) * lambda_cycle
        
        loss_G = loss_GAN_G + loss_cycle_backward_G + loss_identity_G
        loss_F = loss_GAN_F + loss_cycle_forward_F + loss_identity_F
        
        loss_G.backward()
        loss_F.backward()
        
        optimizer_G.step()
        optimizer_F.step()
        
        # 记录损失
        total_loss_G += loss_G.item()
        total_loss_F += loss_F.item()
        total_loss_D_photo += loss_D_photo.item()
        total_loss_D_label += loss_D_label.item()
        total_loss_cycle += (loss_cycle_forward.item() + loss_cycle_backward.item())
        num_batches += 1
        
        cycle_loss_total = loss_cycle_forward.item() + loss_cycle_backward.item()
        pbar.set_postfix({
            "G": f"{loss_G.item():.4f}",
            "F": f"{loss_F.item():.4f}",
            "D_photo": f"{loss_D_photo.item():.4f}",
            "D_label": f"{loss_D_label.item():.4f}",
            "Cycle": f"{cycle_loss_total:.4f}",
        })
    
    avg_loss_G = total_loss_G / num_batches if num_batches > 0 else 0.0
    avg_loss_F = total_loss_F / num_batches if num_batches > 0 else 0.0
    avg_loss_D_photo = total_loss_D_photo / num_batches if num_batches > 0 else 0.0
    avg_loss_D_label = total_loss_D_label / num_batches if num_batches > 0 else 0.0
    avg_loss_cycle = total_loss_cycle / num_batches if num_batches > 0 else 0.0
    
    return {
        "loss_G": avg_loss_G,
        "loss_F": avg_loss_F,
        "loss_D_photo": avg_loss_D_photo,
        "loss_D_label": avg_loss_D_label,
        "loss_cycle": avg_loss_cycle,
    }


@torch.no_grad()
def validate(
    generator_G, generator_F,
    dataloader, l1_loss, device, epoch, output_dir,
    num_samples=10
):
    """验证并保存三联图"""
    generator_G.eval()
    generator_F.eval()
    
    total_loss_cycle = 0.0
    total_metrics = {"psnr": 0.0, "ssim": 0.0, "mae": 0.0}
    num_batches = 0
    saved_samples = 0
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch} [Val]")
    for batch_idx, batch in enumerate(pbar):
        label = batch["label"].to(device)
        photo = batch["photo"].to(device)
        name = batch["name"]
        
        fake_photo = generator_G(label)
        fake_label = generator_F(photo)
        
        recovered_label = generator_F(fake_photo)
        recovered_photo = generator_G(fake_label)
        
        loss_cycle_forward = l1_loss(recovered_label, label)
        loss_cycle_backward = l1_loss(recovered_photo, photo)
        loss_cycle = loss_cycle_forward + loss_cycle_backward
        
        total_loss_cycle += loss_cycle.item()
        
        metrics = evaluate_batch(fake_photo, photo)
        for key in total_metrics:
            total_metrics[key] += metrics[key]
        
        num_batches += 1
        
        if saved_samples < num_samples:
            save_path = output_dir / "images" / f"epoch_{epoch:03d}_sample_{saved_samples:02d}_label2photo.png"
            save_path.parent.mkdir(parents=True, exist_ok=True)
            save_triplet(label, fake_photo, photo, save_path, "Label | Generated Photo | Ground Truth Photo")
            
            save_path = output_dir / "images" / f"epoch_{epoch:03d}_sample_{saved_samples:02d}_photo2label.png"
            save_triplet(photo, fake_label, label, save_path, "Photo | Generated Label | Ground Truth Label")
            
            save_path = output_dir / "images" / f"epoch_{epoch:03d}_sample_{saved_samples:02d}_cycle.png"
            save_cycle_triplet(
                label, fake_photo, recovered_label,
                photo, fake_label, recovered_photo,
                save_path
            )
            
            saved_samples += 1
        
        pbar.set_postfix({
            "cycle_loss": f"{loss_cycle.item():.4f}",
            "psnr": f"{metrics['psnr']:.2f}",
        })
    
    avg_loss_cycle = total_loss_cycle / num_batches if num_batches > 0 else 0.0
    avg_metrics = {key: val / num_batches for key, val in total_metrics.items()}
    
    return avg_loss_cycle, avg_metrics


print("✅ 训练和验证函数定义完成！")


## 7. 开始训练

**注意**：训练可能需要较长时间，建议：
- 使用GPU运行时
- 定期保存checkpoint（每10个epoch）
- 如果Colab会话断开，可以从checkpoint恢复训练

### Checkpoint管理说明

**保存位置**：
- Checkpoint自动保存到Google Drive的 `cyclegan_checkpoints` 文件夹
- 路径：`/content/drive/MyDrive/cyclegan_checkpoints/`

**恢复训练**：
1. 在下面的代码中，找到 `RESUME_EPOCH` 参数
2. 设置为要恢复的epoch编号（例如：`RESUME_EPOCH = 20`）
3. 确保对应的checkpoint文件已存在于Google Drive中

**Checkpoint文件说明**：
- 每个checkpoint文件包含完整的模型状态（生成器、判别器、优化器、学习率调度器等）
- 通常只需要保留最新的checkpoint（如epoch 20），因为可以从任意checkpoint恢复
- 文件名格式：`checkpoint_epoch_020.pth`（3位数字，补零）


In [ ]:
# ============================================
# 训练配置
# ============================================
epochs = 200
batch_size = 1  # CycleGAN论文设置
lr = 2e-4
start_decay_epoch = 100
lambda_cycle = 10.0
lambda_identity = 0.5
n_residual_blocks = 9

# 创建模型
print("创建模型...")
generator_G = CycleGANGenerator(
    in_channels=3,
    out_channels=3,
    n_residual_blocks=n_residual_blocks
).to(device)
generator_F = CycleGANGenerator(
    in_channels=3,
    out_channels=3,
    n_residual_blocks=n_residual_blocks
).to(device)
discriminator_photo = CycleGANDiscriminator(in_channels=3).to(device)
discriminator_label = CycleGANDiscriminator(in_channels=3).to(device)

# 损失函数和优化器
gan_loss = GANLoss()
l1_loss = nn.L1Loss()

optimizer_G = optim.Adam(generator_G.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_F = optim.Adam(generator_F.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_photo = optim.Adam(discriminator_photo.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_label = optim.Adam(discriminator_label.parameters(), lr=lr, betas=(0.5, 0.999))

# 学习率调度
def get_lr_scheduler(optimizer, num_epochs, start_decay_epoch):
    def lr_lambda(epoch):
        if epoch < start_decay_epoch:
            return 1.0
        else:
            return 1.0 - (epoch - start_decay_epoch) / (num_epochs - start_decay_epoch)
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

scheduler_G = get_lr_scheduler(optimizer_G, epochs, start_decay_epoch)
scheduler_F = get_lr_scheduler(optimizer_F, epochs, start_decay_epoch)
scheduler_D_photo = get_lr_scheduler(optimizer_D_photo, epochs, start_decay_epoch)
scheduler_D_label = get_lr_scheduler(optimizer_D_label, epochs, start_decay_epoch)

# ============================================
# 恢复训练（从checkpoint继续）
# ============================================
# 指定要恢复的checkpoint路径
# 如果checkpoint在Google Drive中，直接指定文件名即可（会自动在CHECKPOINT_DIR中查找）
# 例如：从epoch 20继续训练
RESUME_EPOCH = 20  # 设置为None表示从头开始，或设置为要恢复的epoch编号
# 或者直接指定checkpoint文件名：
# resume_checkpoint = CHECKPOINT_DIR / "checkpoint_epoch_020.pth"

if RESUME_EPOCH is not None:
    resume_checkpoint = CHECKPOINT_DIR / f"checkpoint_epoch_{RESUME_EPOCH:03d}.pth"
    print(f"📌 准备从epoch {RESUME_EPOCH}恢复训练")
    print(f"   Checkpoint路径: {resume_checkpoint}")
    if not resume_checkpoint.exists():
        print(f"⚠️  警告：checkpoint文件不存在: {resume_checkpoint}")
        print("   请确保：")
        print("   1. Google Drive已正确挂载")
        print("   2. checkpoint文件已上传到Drive的checkpoint目录")
        print("   3. 文件名格式正确（如：checkpoint_epoch_020.pth）")
        resume_checkpoint = None
else:
    resume_checkpoint = None

start_epoch = 1

# 初始化训练历史列表
train_losses_G = []
train_losses_F = []
train_losses_D_photo = []
train_losses_D_label = []
train_losses_cycle = []
val_losses_cycle = []
val_psnrs = []
val_ssims = []
val_maes = []

# 尝试从checkpoint恢复
if resume_checkpoint is not None and Path(resume_checkpoint).exists():
    print(f"🔁 恢复训练自: {resume_checkpoint}")
    checkpoint = torch.load(resume_checkpoint, map_location=device, weights_only=False)
    
    generator_G.load_state_dict(checkpoint["generator_G_state_dict"])
    generator_F.load_state_dict(checkpoint["generator_F_state_dict"])
    discriminator_photo.load_state_dict(checkpoint["discriminator_photo_state_dict"])
    discriminator_label.load_state_dict(checkpoint["discriminator_label_state_dict"])
    optimizer_G.load_state_dict(checkpoint["optimizer_G_state_dict"])
    optimizer_F.load_state_dict(checkpoint["optimizer_F_state_dict"])
    optimizer_D_photo.load_state_dict(checkpoint["optimizer_D_photo_state_dict"])
    optimizer_D_label.load_state_dict(checkpoint["optimizer_D_label_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    
    for _ in range(start_epoch - 1):
        scheduler_G.step()
        scheduler_F.step()
        scheduler_D_photo.step()
        scheduler_D_label.step()
    
    history_path = LOGS_DIR / "history.json"
    if history_path.exists():
        with open(history_path, "r") as f:
            history = json.load(f)
        train_losses_G = history.get("train_losses_G", [])
        train_losses_F = history.get("train_losses_F", [])
        train_losses_D_photo = history.get("train_losses_D_photo", [])
        train_losses_D_label = history.get("train_losses_D_label", [])
        train_losses_cycle = history.get("train_losses_cycle", [])
        val_losses_cycle = history.get("val_losses_cycle", [])
        val_psnrs = history.get("val_psnrs", [])
        val_ssims = history.get("val_ssims", [])
        val_maes = history.get("val_maes", [])
        print(f"✅ 已恢复训练历史（{len(train_losses_G)} 个epoch）")
    
    print(f"📌 将从 Epoch {start_epoch} 继续训练到 Epoch {epochs}")
else:
    print(f"⚠️  未找到checkpoint或未指定checkpoint")
    print("   将从 Epoch 1 开始全新训练")

print(f"\n🚀 Starting CycleGAN training...")
print(f"   Epochs: {epochs}, Batch size: {batch_size}, LR: {lr}")
print(f"   Lambda cycle: {lambda_cycle}, Lambda identity: {lambda_identity}")
print(f"   Residual blocks: {n_residual_blocks}")
print(f"   Checkpoint保存路径: {CHECKPOINT_DIR}")

# 训练循环
best_val_psnr = 0.0

for epoch in range(start_epoch, epochs + 1):
    # 训练
    train_losses = train_epoch(
        generator_G, generator_F,
        discriminator_photo, discriminator_label,
        train_loader, gan_loss, l1_loss,
        optimizer_G, optimizer_F,
        optimizer_D_photo, optimizer_D_label,
        device, epoch,
        lambda_cycle=lambda_cycle,
        lambda_identity=lambda_identity
    )
    
    # 验证
    val_loss_cycle, val_metrics = validate(
        generator_G, generator_F,
        val_loader, l1_loss, device, epoch, OUTPUT_ROOT / "cyclegan",
        num_samples=10
    )
    
    # 学习率调度
    scheduler_G.step()
    scheduler_F.step()
    scheduler_D_photo.step()
    scheduler_D_label.step()
    current_lr = optimizer_G.param_groups[0]["lr"]
    
    # 记录历史
    train_losses_G.append(train_losses["loss_G"])
    train_losses_F.append(train_losses["loss_F"])
    train_losses_D_photo.append(train_losses["loss_D_photo"])
    train_losses_D_label.append(train_losses["loss_D_label"])
    train_losses_cycle.append(train_losses["loss_cycle"])
    val_losses_cycle.append(val_loss_cycle)
    val_psnrs.append(val_metrics["psnr"])
    val_ssims.append(val_metrics["ssim"])
    val_maes.append(val_metrics["mae"])
    
    if epoch % 10 == 0:
        print(f"\nEpoch {epoch}/{epochs}")
        print(f"  Train - G: {train_losses['loss_G']:.4f}, F: {train_losses['loss_F']:.4f}, "
              f"D_photo: {train_losses['loss_D_photo']:.4f}, D_label: {train_losses['loss_D_label']:.4f}, "
              f"Cycle: {train_losses['loss_cycle']:.4f}")
        print(f"  Val - Cycle Loss: {val_loss_cycle:.4f}, PSNR: {val_metrics['psnr']:.2f}dB, "
              f"SSIM: {val_metrics['ssim']:.4f}, MAE: {val_metrics['mae']:.4f}, LR: {current_lr:.6f}")
        
        # 保存checkpoint
        torch.save({
            "epoch": epoch,
            "generator_G_state_dict": generator_G.state_dict(),
            "generator_F_state_dict": generator_F.state_dict(),
            "discriminator_photo_state_dict": discriminator_photo.state_dict(),
            "discriminator_label_state_dict": discriminator_label.state_dict(),
            "optimizer_G_state_dict": optimizer_G.state_dict(),
            "optimizer_F_state_dict": optimizer_F.state_dict(),
            "optimizer_D_photo_state_dict": optimizer_D_photo.state_dict(),
            "optimizer_D_label_state_dict": optimizer_D_label.state_dict(),
            "scheduler_G_state_dict": scheduler_G.state_dict(),
            "scheduler_F_state_dict": scheduler_F.state_dict(),
            "scheduler_D_photo_state_dict": scheduler_D_photo.state_dict(),
            "scheduler_D_label_state_dict": scheduler_D_label.state_dict(),
            "val_psnr": val_metrics["psnr"],
            "val_ssim": val_metrics["ssim"],
            "val_mae": val_metrics["mae"],
        }, CHECKPOINT_DIR / f"checkpoint_epoch_{epoch:03d}.pth")
        
        # 保存训练历史
        history = {
            "train_losses_G": train_losses_G,
            "train_losses_F": train_losses_F,
            "train_losses_D_photo": train_losses_D_photo,
            "train_losses_D_label": train_losses_D_label,
            "train_losses_cycle": train_losses_cycle,
            "val_losses_cycle": val_losses_cycle,
            "val_psnrs": val_psnrs,
            "val_ssims": val_ssims,
            "val_maes": val_maes,
        }
        with open(LOGS_DIR / "history.json", "w") as f:
            json.dump(history, f, indent=2)
        
        checkpoint_path = CHECKPOINT_DIR / f"checkpoint_epoch_{epoch:03d}.pth"
        print(f"✅ Checkpoint已保存: {checkpoint_path}")
        
        # 如果保存到Drive，提示用户
        if IN_COLAB and USE_DRIVE_FOR_CHECKPOINTS and DRIVE_ROOT:
            print(f"   💾 已保存到Google Drive，即使会话断开也不会丢失")

# 保存最终模型
torch.save({
    "generator_G_state_dict": generator_G.state_dict(),
    "generator_F_state_dict": generator_F.state_dict(),
    "discriminator_photo_state_dict": discriminator_photo.state_dict(),
    "discriminator_label_state_dict": discriminator_label.state_dict(),
}, CHECKPOINT_DIR / "model_final.pth")

print(f"\n✅ Training complete!")
print(f"   Final Val Cycle Loss: {val_losses_cycle[-1]:.4f}")
print(f"   Final PSNR: {val_psnrs[-1]:.2f}dB")
print(f"   Final SSIM: {val_ssims[-1]:.4f}")
print(f"   Final MAE: {val_maes[-1]:.4f}")
print(f"\n📁 所有结果已保存到: {OUTPUT_ROOT}")
print(f"📁 Checkpoint保存在: {CHECKPOINT_DIR}")
if IN_COLAB and USE_DRIVE_FOR_CHECKPOINTS and DRIVE_ROOT:
    print(f"   ✅ Checkpoint已保存到Google Drive，即使Colab会话断开也不会丢失")
    print(f"   💡 下次训练时，设置 RESUME_EPOCH = {epochs} 即可从此checkpoint继续训练")


In [ ]:
# 绘制训练损失曲线
if len(train_losses_G) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    epochs_range = range(1, len(train_losses_G) + 1)
    
    # 生成器损失
    axes[0, 0].plot(epochs_range, train_losses_G, label="Generator G", color="blue")
    axes[0, 0].plot(epochs_range, train_losses_F, label="Generator F", color="green")
    axes[0, 0].set_xlabel("Epoch")
    axes[0, 0].set_ylabel("Loss")
    axes[0, 0].set_title("Generator Losses")
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 判别器损失
    axes[0, 1].plot(epochs_range, train_losses_D_photo, label="Discriminator Photo", color="red")
    axes[0, 1].plot(epochs_range, train_losses_D_label, label="Discriminator Label", color="orange")
    axes[0, 1].set_xlabel("Epoch")
    axes[0, 1].set_ylabel("Loss")
    axes[0, 1].set_title("Discriminator Losses")
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 循环一致性损失
    axes[0, 2].plot(epochs_range, train_losses_cycle, label="Train Cycle Loss", color="purple")
    axes[0, 2].plot(epochs_range, val_losses_cycle, label="Val Cycle Loss", color="magenta", linestyle="--")
    axes[0, 2].set_xlabel("Epoch")
    axes[0, 2].set_ylabel("Loss")
    axes[0, 2].set_title("Cycle Consistency Loss")
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # PSNR
    axes[1, 0].plot(epochs_range, val_psnrs, label="PSNR", color="blue")
    axes[1, 0].set_xlabel("Epoch")
    axes[1, 0].set_ylabel("PSNR (dB)")
    axes[1, 0].set_title("Validation PSNR")
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # SSIM
    axes[1, 1].plot(epochs_range, val_ssims, label="SSIM", color="green")
    axes[1, 1].set_xlabel("Epoch")
    axes[1, 1].set_ylabel("SSIM")
    axes[1, 1].set_title("Validation SSIM")
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # MAE
    axes[1, 2].plot(epochs_range, val_maes, label="MAE", color="red")
    axes[1, 2].set_xlabel("Epoch")
    axes[1, 2].set_ylabel("MAE")
    axes[1, 2].set_title("Validation MAE")
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / "training_curves.png", dpi=150, bbox_inches="tight")
    plt.show()
else:
    print("请先完成训练")


In [ ]:
# 加载保存的三联图并展示
import glob

sample_images_label2photo = sorted(glob.glob(str(IMAGES_DIR / "epoch_*_sample_*_label2photo.png")))
sample_images_cycle = sorted(glob.glob(str(IMAGES_DIR / "epoch_*_sample_*_cycle.png")))

if sample_images_label2photo:
    # 显示Label→Photo方向的样例
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle("CycleGAN Generated Samples (Label → Photo)", fontsize=16)
    
    for idx, img_path in enumerate(sample_images_label2photo[:6]):
        row = idx // 3
        col = idx % 3
        img = Image.open(img_path)
        axes[row, col].imshow(img)
        axes[row, col].set_title(f"Sample {idx + 1}")
        axes[row, col].axis("off")
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / "samples_label2photo.png", dpi=150, bbox_inches="tight")
    plt.show()
    
    # 显示循环一致性结果
    if sample_images_cycle:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle("CycleGAN Cycle Consistency Results", fontsize=16)
        
        for idx, img_path in enumerate(sample_images_cycle[:6]):
            row = idx // 3
            col = idx % 3
            img = Image.open(img_path)
            axes[row, col].imshow(img)
            axes[row, col].set_title(f"Sample {idx + 1}")
            axes[row, col].axis("off")
        
        plt.tight_layout()
        plt.savefig(FIGURES_DIR / "samples_cycle.png", dpi=150, bbox_inches="tight")
        plt.show()
else:
    print("未找到样例图像。请先运行训练和验证。")


## 9. 下载结果（可选）

训练完成后，可以将结果下载到本地：


In [ ]:
# 如果结果保存在Colab本地（/content/outputs），可以使用以下代码下载
# 注意：如果结果已保存到Google Drive，可以直接从Drive下载

if IN_COLAB:
    print("📥 下载结果文件...")
    print(f"结果保存在: {OUTPUT_ROOT}")
    print("\n下载方法：")
    print("1. 使用Colab的文件浏览器（左侧文件图标）")
    print("2. 右键点击文件/文件夹 → 下载")
    print("3. 或使用以下命令创建zip压缩包：")
    print(f"   !zip -r cyclegan_results.zip {OUTPUT_ROOT}")
    
    # 可选：自动创建zip文件
    # import shutil
    # shutil.make_archive("cyclegan_results", "zip", OUTPUT_ROOT)
    # print("✅ 已创建压缩包: cyclegan_results.zip")
else:
    print("本地环境，结果已保存在本地文件系统")
